<h1>The Code</h1>

In [8]:
import geocoder
import requests
import pandas as pd
import numpy as np
import random
from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import folium

print('Folium installed')
print('Libraries imported.')


Folium installed
Libraries imported.


In [9]:
CLIENT_ID = 'T2RR1TSRPCIDSI12ACSA1OXORAUGMNLHNGA53T0FLIDWMVQV'
CLIENT_SECRET = 'YGCD2W5FHF4VPYKGUREIKXB3UOF1JFA42TEDJTJOJMCUQZGQ'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T2RR1TSRPCIDSI12ACSA1OXORAUGMNLHNGA53T0FLIDWMVQV
CLIENT_SECRET:YGCD2W5FHF4VPYKGUREIKXB3UOF1JFA42TEDJTJOJMCUQZGQ


In [10]:
g = geocoder.ip("me")
latitude = g.lat
longitude = g.lng
print(latitude, longitude)

45.5946 -121.1787


In [16]:
search_query = 'restaurant'
radius = 500
print(search_query + ' .... OK!')

restaurant .... OK!


In [17]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=T2RR1TSRPCIDSI12ACSA1OXORAUGMNLHNGA53T0FLIDWMVQV&client_secret=YGCD2W5FHF4VPYKGUREIKXB3UOF1JFA42TEDJTJOJMCUQZGQ&ll=45.5946,-121.1787&v=20180604&query=restaurant&radius=500&limit=30'

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f10bed80af06e78b973fd7d'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d116941735',
      'name': 'Bar',
      'pluralName': 'Bars',
      'primary': True,
      'shortName': 'Bar'}],
    'hasPerk': False,
    'id': '4b8f41d1f964a5206e4f33e3',
    'location': {'address': '604 E 2nd St',
     'cc': 'US',
     'city': 'The Dalles',
     'country': 'United States',
     'distance': 522,
     'formattedAddress': ['604 E 2nd St',
      'The Dalles, OR 97058',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.599276296574764,
       'lng': -121.17810486842023},
      {'label': 'entrance', 'lat': 45.599124, 'lng': -121.17822}],
     'lat': 45.599276296574764,
     'lng': -121.17810486842023,
     'postalCode': '97058',
     'state': 'OR'},
    'name': "Zim's Brau Haus Restaurant & Sports Bar",
 

In [19]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4b8f41d1f964a5206e4f33e3,Zim's Brau Haus Restaurant & Sports Bar,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1594932987,False,604 E 2nd St,45.599276,-121.178105,"[{'label': 'display', 'lat': 45.59927629657476...",522,97058,US,The Dalles,OR,United States,"[604 E 2nd St, The Dalles, OR 97058, United St...",481186646


In [20]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Zim's Brau Haus Restaurant & Sports Bar,Bar,604 E 2nd St,45.599276,-121.178105,"[{'label': 'display', 'lat': 45.59927629657476...",522,97058,US,The Dalles,OR,United States,"[604 E 2nd St, The Dalles, OR 97058, United St...",4b8f41d1f964a5206e4f33e3


In [21]:
dataframe_filtered.name

0    Zim's Brau Haus Restaurant & Sports Bar
Name: name, dtype: object

In [22]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map